In [1]:
%load_ext tensorboard
import pathlib

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import tensorflow as tf
import datetime
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 
from IPython.display import clear_output


print(tf.__version__)

2.3.1


In [2]:

dataset = pd.read_csv('train.csv',encoding="utf8")



In [3]:
dataset = dataset.fillna(1,axis=1)
cols = list(dataset.columns)
dataset[cols] = dataset[cols].apply(pd.to_numeric,errors='coerce')
#dataset = dataset.apply()

#dataset = pd.to_numeric(dataset)
dataset = dataset.astype(int)
dataset['Энергопотребление (анализируемый показатель), млн. кВт в час']
len(list(dataset.columns))
dataset.isnull().values.any()

False

In [4]:
train_dataset = dataset.sample(frac=0.7,random_state=0)
test_dataset = dataset.drop(train_dataset.index)
#list(train_dataset.columns)
#train_dataset.to_csv("test.csv")

In [5]:
train_labels = train_dataset.pop('Энергопотребление (анализируемый показатель), млн. кВт в час')
test_labels = test_dataset.pop('Энергопотребление (анализируемый показатель), млн. кВт в час')
train_labels

1895    2796
572     2796
1368    2796
3175    2796
1732    2796
        ... 
79      2796
662     2796
1439    2796
3557    2796
919     2796
Name: Энергопотребление (анализируемый показатель), млн. кВт в час, Length: 2557, dtype: int64

In [6]:
#list(train_dataset.columns)

train_stats = train_dataset.describe()
#train_stats.pop('Энергопотребление (анализируемый показатель), млн. кВт в час')
train_stats = train_stats.transpose()
train_stats

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,2557.0,1815.473211,1062.912804,1.0,904.0,1802.0,2734.0,3652.0
"ВРП, рублей",2557.0,68570.544779,29095.786823,0.0,57815.0,68280.0,89231.0,108882.0
Белгородская область,2557.0,1004.126711,411.706409,0.0,906.0,1010.0,1287.0,1532.0
Брянская область,2557.0,477.988659,207.714778,0.0,375.0,481.0,629.0,747.0
Владимирская область,2557.0,584.037153,241.398632,0.0,498.0,592.0,774.0,879.0
...,...,...,...,...,...,...,...,...
Амурская область.9,2557.0,19.000000,0.000000,19.0,19.0,19.0,19.0,19.0
Магаданская область.8,2557.0,5.000000,0.000000,5.0,5.0,5.0,5.0,5.0
Сахалинская область.8,2557.0,10.000000,0.000000,10.0,10.0,10.0,10.0,10.0
Еврейская автономная область.7,2557.0,3.000000,0.000000,3.0,3.0,3.0,3.0,3.0


In [7]:
#def norm(x):
#  return (x - train_stats['mean']) / train_stats['std']
#normed_train_data = norm(train_dataset)
#normed_test_data = norm(test_dataset)
#normed_train_data

In [8]:
def build_model():
  model = keras.Sequential([
    layers.Dense(128,kernel_initializer='normal',input_shape=[len(train_dataset.keys())]),
    layers.Dense(256,activation='relu',kernel_initializer='normal'),
    layers.Dense(256,activation='relu',kernel_initializer='normal'),
    layers.Dense(256,activation='relu',kernel_initializer='normal'),
    layers.Dense(1,kernel_initializer='normal',activation='linear')
  ])
  
    
  optimizer = tf.keras.optimizers.RMSprop(0.1)

  model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
  return model


In [9]:
model = build_model()

In [10]:
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               106368    
_________________________________________________________________
dense_1 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_3 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 257       
Total params: 271,233
Trainable params: 271,233
Non-trainable params: 0
_________________________________________________________________


In [11]:
example_batch = train_dataset[:10]
#example_batch
example_result = model.predict(example_batch)
example_result

array([[-331.61993],
       [-237.3082 ],
       [-269.51337],
       [-347.62335],
       [-297.3246 ],
       [-202.40991],
       [-347.65106],
       [-237.00539],
       [-298.19376],
       [-347.6545 ]], dtype=float32)

In [12]:
class PrintDot(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
        clear_output(wait=True)
        print(str(epoch)+","+'loss' + str(logs))

EPOCHS = 500

In [ ]:
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
log_dir =  datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
history = model.fit(train_dataset, train_labels, epochs=EPOCHS,
                    validation_split = 0.2, verbose=0, callbacks=[tensorboard_callback,PrintDot()])


311,loss{'loss': 31.916627883911133, 'mean_absolute_error': 31.916627883911133, 'val_loss': 12.350423812866211, 'val_mean_absolute_error': 12.350423812866211}


In [81]:
#%tensorboard --logdir .

Reusing TensorBoard on port 6007 (pid 11385), started 1:15:26 ago. (Use '!kill 11385' to kill it.)

In [27]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()


,loss,mean_absolute_error,val_loss,val_mean_absolute_error,epoch
495,10.867300,10.867300,12.524193,12.524193,495
496,10.813380,10.813380,18.905186,18.905186,496
497,14.126811,14.126811,21.281818,21.281818,497
498,14.061516,14.061516,10.294369,10.294369,498
499,16.300026,16.300026,26.337448,26.337448,499


In [28]:
train_labels

1895    2796
572     2796
1368    2796
3175    2796
1732    2796
        ... 
79      2796
662     2796
1439    2796
3557    2796
919     2796
Name: Энергопотребление (анализируемый показатель), млн. кВт в час, Length: 2557, dtype: int64

In [29]:
test_predictions = model.predict(test_dataset).flatten()

test_predictions
#mean_absolute_error(train_labels, test_predictions)

array([ 919.1826, 2809.834 , 2809.764 , ..., 2824.088 , 2824.0764,
       2824.064 ], dtype=float32)